In [26]:
import sys
import os
import cv2 as cv
import numpy as np
from math import log10, sqrt 

# Global Variables
DELAY_CAPTION = 1500
DELAY_BLUR = 100
MAX_KERNEL_LENGTH = 31

src = None
dst = None
window_name = 'Smoothing Demo'

def main():
    cv.namedWindow(window_name, cv.WINDOW_AUTOSIZE)

    # Load the source image
    imageName = 'lena.jpg'

    # Current working directory
    script_dir = os.getcwd()

    # Construct full path to image
    image_path = os.path.join(script_dir, imageName)

    global src
    src = cv.imread(image_path)
    
    if src is None:
        print('Error opening image')
        print('Usage: smoothing.py [image_name -- default ../data/lena.jpg] \n')
        return -1

    if display_caption('Original Image') != 0:
        return 0
    
   

    global dst
    psnr_value = PSNR(src, dst)
    dst = np.copy(src)
    if display_dst(DELAY_CAPTION) != 0:
        return 0

    # Median blur
    if display_caption('Median Blur') != 0:
        return 0
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.medianBlur(src, i)
        if display_dst(DELAY_BLUR) != 0:
            return 0
    medianPsnr = PSNR(src, dst)
    print('PSNR for Median Blur: '+ str(medianPsnr))

    # Mean blur
    if display_caption('Mean Blur') != 0:
        return 0
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.blur(src, (i, i))
        if display_dst(DELAY_BLUR) != 0:
            return 0
    meanPsnr = PSNR(src, dst)
    print('PSNR for Mean Blur: '+ str(meanPsnr))

    # Bilateral Filter
    if display_caption('Bilateral Blur') != 0:
        return 0
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.bilateralFilter(src, i, i * 2, i / 2)
        if display_dst(DELAY_BLUR) != 0:
            return 0
    bilateralPsnr = PSNR(src, dst)
    print('PSNR for Bilateral Filter: '+ str(bilateralPsnr))

    # Gaussian blur
    if display_caption('Gaussian Blur') != 0:
        return 0
    for i in range(1, MAX_KERNEL_LENGTH, 2):
        dst = cv.GaussianBlur(src, (i, i), 0)
        if display_dst(DELAY_BLUR) != 0:
            return 0
    gaussianPsnr = PSNR(src, dst)
    print('PSNR for Gaussian Blur: '+ str(gaussianPsnr))
    
    #  Done
    display_caption('Done!')
    return 0

   
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
# Based from : https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/

def display_caption(caption):
    global dst
    dst = np.zeros(src.shape, src.dtype)
    rows, cols, _ch = src.shape
    cv.putText(dst, caption,
                (int(cols / 4), int(rows / 2)),
                cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255))

    return display_dst(DELAY_CAPTION)

def display_dst(delay):
    cv.imshow(window_name, dst)
    c = cv.waitKey(delay)
    if c >= 0 : return -1
    return 0

if __name__ == "__main__":
    main()
    

PSNR for Median Blur: 31.173125766944484
PSNR for Mean Blur: 29.873956847181216
PSNR for Bilateral Filter: 33.126306280828665
PSNR for Gaussian Blur: 31.147064160112127


 The Bilateral Filter method produced the highest Peak Signal-to-Noise Ratio (PSNR) amongst the four methods. Since the Bilateral Filter is the only method used that takes into consideration the difference in pixel intensities, this lets it keep more details when it comes to areas in the image where there is huge contrast between neighboring pixels; thus preventing the blurring of the image.